In [ ]:
import os
import pandas as pd
import pandas as pd
import numpy as np
import cvxpy as cp
import yfinance as yf
from tqdm import tqdm
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import gurobipy as gp
from gurobipy import GRB
from scipy.stats import norm

In [ ]:
# Load the stock data.
file_path = '..\data\s&p500\s&p500_2016to2022_stock_data.csv'
stock_data = pd.read_csv(file_path, index_col='date', parse_dates=True)

In [ ]:
stock_data.head()

In [ ]:
# Calculate daily returns
stock_returns = stock_data.pct_change().dropna(how='all')
stock_returns.tail()

In [ ]:
stock_returns['^GSPC'].isna().sum()


# Experiment setup
- daily return을 기준으로 index tracking 설정
- 월별 리벨런싱 가정
- 2006/01/01 ~ 2022-12/30
- k act: 10% 20% 30% 40% 50% - 전체 주식의 % 

# Algorithm

### Gurobi with adaptive loss 예제

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# 데이터 초기화
max_iterations = 10  # 최대 반복 횟수
epsilon = 1e-6  # 수렴 조건
w = 1.0  # 초기 가중치
previous_obj_value = float('inf')  # 초기 Objective Function 값

# 모델 생성
model = gp.Model("adaptive_weights")

# 변수 추가
x = model.addVar(name="x")
y = model.addVar(name="y")

# 제약 조건 추가
model.addConstr(x + 2 * y <= 4, "c0")
model.addConstr(2 * x + y <= 5, "c1")

# 반복 프로세스
for iteration in range(max_iterations):
    # 목표 함수 설정
    model.setObjective(w * (x + y), GRB.MAXIMIZE)
    
    # 최적화 수행
    model.optimize()
    
    # 결과 출력
    if model.status == GRB.OPTIMAL or model.status == GRB.SUBOPTIMAL:
        for v in model.getVars():
            print(f'Iteration {iteration}, {v.varName}: {v.x}')
        print(f'Iteration {iteration}, Obj: {model.objVal}')
        
        # 가중치 업데이트 (예: 목표 함수 값의 절대값을 가중치로 사용)
        new_w = abs(model.objVal)
        
        # 수렴 확인
        if abs(previous_obj_value - model.objVal) < epsilon:
            print(f"Converged at iteration {iteration}")
            break
        
        # 업데이트된 가중치 및 이전 목표 함수 값 갱신
        w = new_w
        previous_obj_value = model.objVal
    else:
        print(f"Optimization was stopped at iteration {iteration}")
        break

# 최종 결과 출력
if model.status == GRB.OPTIMAL or model.status == GRB.SUBOPTIMAL:
    for v in model.getVars():
        print(f'Final {v.varName}: {v.x}')
    print(f'Final Obj: {model.objVal}')
else:
    print("최적 해를 찾지 못했습니다.")


## CARD

In [ ]:
def card(X, y, kact, time_limit=1200):
    p = X.shape[1]
    
    # Define variables
    beta = cp.Variable(p)
    z = cp.Variable(p, boolean=True)  # Binary variables for cardinality constraint
    
    # Objective function
    objective = cp.Minimize(cp.norm2(y - X @ beta) ** 2)
    
    # Constraints
    constraints = [
        cp.sum(beta) == 1,  # Sum of weights is 1
        beta >= 0,  # All weights beta are non-negative
        cp.sum(z) <= kact,  # Number of non-zero weights is less than or equal to kact
        beta <= z  # Link binary variables with beta
    ]
    
    # Problem definition
    prob = cp.Problem(objective, constraints)
    
    # Solve the problem
    prob.solve(solver=cp.GUROBI, TimeLimit=time_limit)
    
    beta_opt = beta.value
    
    return beta_opt

In [ ]:
stock_stocks = stock_returns.drop(columns=['^GSPC'])
stock_index = stock_returns['^GSPC']

train_data = stock_stocks.iloc[:250]

# Filter out columns with NaN values in the current window
valid_assets_train = train_data.dropna(axis=1)
valid_assets = valid_assets_train.columns.intersection(valid_assets_train.columns)

X_train = valid_assets_train[valid_assets].values
y_train = stock_index.iloc[:250].values

In [ ]:
# # CARD
beta_card = card(X_train, y_train, kact=10, time_limit=60)
non_negative_count = np.sum(beta_card > 0)


In [ ]:
non_negative_count

## ALASSO Algorithm
- make initial beta using OLS
- can not find propal lambda in the context
- FIX!: ALASSO solution is always converge to 1,0,0,0 -> shrinking is too strong

In [ ]:
from sklearn.linear_model import Lasso

def alasso(X, y, kact, time_limit=1200, lambdas=1):
    p = X.shape[1]
    
    # Initial Lasso to get initial weights
    lasso = Lasso(alpha=0.1, max_iter=10000)
    lasso.fit(X, y)
    beta_init = lasso.coef_
    
    # Adaptive weights
    weights = 1 / (np.abs(beta_init) + 1e-6)
    
    # Define variables
    beta = cp.Variable(p)
    z = cp.Variable(p, boolean=True)  # Binary variables for cardinality constraint
    
    # Objective function
    objective = cp.Minimize(cp.norm2(y - X @ beta) ** 2 + lambdas * cp.sum(weights @ cp.abs(beta)))
    
    # Constraints
    constraints = [
        cp.sum(beta) == 1,  # Sum of weights is 1
        beta >= 0,  # All weights beta are non-negative
        cp.sum(z) == kact,  # Number of non-zero weights is less than or equal to kact
        beta <= z  # Link binary variables with beta
    ]
    
    # Problem definition
    prob = cp.Problem(objective, constraints)
    
    # Solve the problem
    prob.solve(solver=cp.GUROBI, TimeLimit=time_limit)
    
    beta_opt = beta.value
    
    return beta_opt


In [ ]:
# ALASSO
beta_alasso = alasso(X_train, y_train, kact=10, time_limit=60)
non_negative_count = np.sum(beta_alasso > 0)
non_negative_count

In [ ]:
non_negative_count

## Sorted LASSO- SLOPE
detail is from Sparse index clones via the sorted l1-Norm
SLOPE—ADAPTIVE VARIABLE SELECTION VIA CONVEX OPTIMIZATION

- initial guess beta를 가지고 추정 시작
- 이후 update
- lambda of model: aplha(=sigma of model)(1-i*theta)~ 



In [ ]:
# Generate the lambda sequence
def new_lambda(beta_value, alpha, theta, p):
    # Compute the ranks of the absolute values of beta_value
    ranks = np.argsort(np.argsort(-np.abs(beta_value))) + 1
    lambdas = np.array([alpha * norm.ppf(1 - (rank * theta / (2 * p))) for rank in ranks])
    # lambdas = np.array([alpha * norm.ppf(1 - (i * theta / (2 * p))) for i in range(1, p + 1)])
    return lambdas  

In [ ]:
def slope(X, y, kact, alpha=1.0, theta=0.1, tol=1e-06, max_iter=10, time_limit=1200, add_loss=0, seed=42):
    """
    Solve the SLOPE optimization problem using Gurobi with an activation constraint.
    
    Parameters:
    X : array-like, shape (T, K)
        The return matrix of benchmark constituents.
    y : array-like, shape (T,)
        The benchmark returns.
    kact : int
        Number of active variables (non-zero coefficients) allowed.
    alpha : float, optional
        Regularization parameter for the lambda sequence.
    theta : float, optional
        Decay rate for the lambda sequence.
    """
    # Number of samples and features
    n, p = X.shape
    weights = np.ones(p)
    m = gp.Model()
    m.setParam('TimeLimit', time_limit)
    m.setParam(GRB.Param.Seed, seed)
    beta = m.addMVar(shape=p, lb=0, name="beta")
    z = m.addMVar(shape=p, vtype=GRB.BINARY, name="z")
    m.addConstr(beta <= z, name="c0")
    m.addConstr(beta.sum() == 1, name="c1")
    m.addConstr(z.sum() <= kact, name="c2")
    for _ in range(max_iter):
        m.reset()   
        objective = (1/n) * ((y - X @ beta) @ (y - X @ beta)) + weights @ beta + add_loss
        m.setObjective(objective, GRB.MINIMIZE)
        m.optimize()
        # 결과 출력
        if m.status == GRB.OPTIMAL or m.status == GRB.SUBOPTIMAL:
            beta_value = beta.X
            new_weights = np.array(new_lambda(beta_value, alpha, theta, p))
            weights = new_weights
            
            if np.linalg.norm(new_weights - weights) < tol:
                break
        else:
            beta_value = np.zeros(p)
            continue
    m.dispose()        
    return beta_value

In [ ]:
# SLOPE
beta_slope = slope(X_train, y_train, kact=20, time_limit=60)

In [ ]:
non_negative_count = np.sum(beta_slope > 0)
non_negative_count

# SLOPE-SLC
1. Sorted LASSO의 변형 형태
2. compute for each group the median partial correlation of consituents and keep only groups which are including 75th percent quantile for the equity

In [ ]:
from sklearn.linear_model import LassoLarsIC

In [ ]:
# def compute_partial_correlation(X, y):
#     # Use LassoLarsIC to find significant features
#     model = LassoLarsIC(criterion='bic')
#     model.fit(X, y)
#     active_features = np.where(model.coef_ != 0)[0]
    
#     partial_corr = np.zeros(X.shape[1])
#     for i in active_features:
#         residual_y = y - model.predict(X[:, active_features])
#         residual_x = X[:, i] - model.predict(X[:, active_features])
#         partial_corr[i] = np.corrcoef(residual_y, residual_x)[0, 1]
    
#     return partial_corr


In [ ]:
# def slope_slc(X, y, kact, alpha=1.0, theta=0.1, tol=1e-06, max_iter=10, time_limit=1200, seed=42):
    
#     # Step 1: Solve SLOPE
#     beta_initial = slope(X, y, alpha, theta, tol, max_iter, time_limit, seed)
    
    
#     # Step 2: Compute partial correlation for each asset
#     partial_corr = compute_partial_correlation(X, y)
    
#     # Step 3: Select groups based on median partial correlation
#     median_corr = np.median(partial_corr)
#     threshold = np.percentile(partial_corr, 75)
#     active_indices = np.where(partial_corr >= threshold)[0]
    
#     # Step 4: Rescale SLOPE estimates to sum to 1
#     beta_rescaled = np.zeros_like(beta_initial)
#     beta_rescaled[active_indices] = beta_initial[active_indices]
#     beta_rescaled /= np.sum(beta_rescaled)
    
#     # Step 5: Ensure the number of active weights is less than or equal to kact
#     if np.sum(beta_rescaled > 0) > kact:
#         sorted_indices = np.argsort(beta_rescaled)[::-1]
#         beta_rescaled[sorted_indices[kact:]] = 0
#         beta_rescaled /= np.sum(beta_rescaled)
    
#     return beta_rescaled

In [ ]:
# # SLOPE-SLC
# beta_slope_slc = slope_slc(X_train, y_train, kact=20, time_limit=60)

In [ ]:
# non_negative_count = np.sum(beta_slope_slc > 0)
# non_negative_count

## MSW_LASSO
detail is from High-dimensional Sprase index tracking based on a multi-step coonvex optimization approach

In [ ]:
def msw_lasso(X, y, kact, max_iter=10, tol=1e-12, penalty='MCP', a=2.5, time_limit = 1200, add_loss=0, seed=42):
    n, p = X.shape
    weights = np.ones(p)
    
    def p_lambda(beta_j, lam, a):
        #MCP: b, a =2.5
        #SCAD: a=3.7
        #log-M: epsilon = 1.67*10^-5
        #lq: q=0.1
        if penalty == 'MCP':
            print("----------------Penalty: MCP--------------------")
            b = a 
            return lam * beta_j - (beta_j**2) / (2 * b) if beta_j <= b * lam else (b * lam**2) / 2
        elif penalty == 'SCAD':
            print("----------------Penalty: SCAD--------------------")
            if beta_j <= lam:
                return lam * beta_j
            elif beta_j <= a * lam:
                return (-beta_j**2 + 2 * a * lam * beta_j - lam**2) / (2 * (a - 1))
            else:
                return (a + 1) * lam**2 / 2
        elif penalty == 'log-M':
            print("----------------Penalty: log-M--------------------")
            eps = a
            return lam * np.log(1+np.abs(beta_j)/eps) / np.log(1+(1/eps))
        elif penalty == 'lq':
            print("----------------Penalty: lq--------------------")
            q = a 
            return lam * (abs(beta_j) **q) 
    m = gp.Model()
    m.setParam('TimeLimit', time_limit)
    m.setParam(GRB.Param.Seed, seed)
    beta = m.addMVar(shape=p, lb=0, name="beta")
    z = m.addMVar(shape=p, vtype=GRB.BINARY, name="z")
    m.addConstr(beta <= z, name="c0")
    m.addConstr(beta.sum() == 1, name="c1")
    m.addConstr(z.sum() <= kact, name="c2")
    
    for _ in range(max_iter):
        m.reset()   
        objective = (1/n) * ((y - X @ beta) @ (y - X @ beta)) + weights @ beta + add_loss
        m.setObjective(objective, GRB.MINIMIZE)
        m.optimize()
        # 결과 출력
        if m.status == GRB.OPTIMAL or m.status == GRB.SUBOPTIMAL:
            beta_value = beta.X
            new_weights = np.array([abs(p_lambda(beta_value[j], 1, a)) for j in range(p)])
            weights = new_weights
            
            if np.linalg.norm(new_weights - weights) < tol:
                break
        else:
            beta_value = np.zeros(p)
            continue
    m.dispose()         
    return beta_value

In [ ]:
# # msw_lasso
beta_msw_lasso = msw_lasso(X_train, y_train, kact=20, penalty='MCP', a=2.5, max_iter=3, time_limit=60)

In [ ]:
non_negative_count = np.sum(beta_msw_lasso > 0)
non_negative_count

In [26]:
beta_msw_lasso = msw_lasso(X_train, y_train, kact=10, penalty='SCAD', a=3.7, max_iter=3, time_limit=60)


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    4251    9.9967097e-01   5.808229e-02   0.000000e+00      5s
    8760    1.0000001e+00   0.000000e+00   0.000000e+00      7s

Root relaxation: objective 1.000000e+00, 8760 iterations, 7.24 seconds (3.96 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.00000    0   90    1.00015    1.00000  0.01%     -    7s
H    0     0                       1.0000167    1.00000  0.00%     -    9s

Explored 1 nodes (8760 simplex iterations) in 9.68 seconds (4.56 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 1.00002 1.00015 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000016680157e+00, best bound 1.000000099790e+00, gap 0.0017%
----------------Penalty: SCAD--------------------
----------------Penalty: SCAD--------------------
----------

In [27]:
non_negative_count = np.sum(beta_msw_lasso > 0)
non_negative_count

12

In [28]:
beta_msw_lasso = msw_lasso(X_train, y_train, kact=10, penalty='log-M', a=1.67*(10**-5), max_iter=3, time_limit=60)

Set parameter TimeLimit to value 60
Set parameter Seed to value 42
Discarded solution information
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 638 rows, 1272 columns and 2544 nonzeros
Model fingerprint: 0x3808f38d
Model has 201894 quadratic objective terms
Variable types: 636 continuous, 636 integer (636 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+01]
  QObjective range [5e-09, 2e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 1.0001486
Presolve time: 0.24s
Presolved: 638 rows, 1272 columns, 2544 nonzeros
Presolved model has 201894 quadratic objectiv

In [29]:
non_negative_count = np.sum(beta_msw_lasso > 0)
non_negative_count

12

In [30]:
beta_msw_lasso = msw_lasso(X_train, y_train, kact=10, penalty='lq', a=0.1, max_iter=3, time_limit=60)

Set parameter TimeLimit to value 60
Set parameter Seed to value 42
Discarded solution information
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 638 rows, 1272 columns and 2544 nonzeros
Model fingerprint: 0x3808f38d
Model has 201894 quadratic objective terms
Variable types: 636 continuous, 636 integer (636 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+01]
  QObjective range [5e-09, 2e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 1.0001486
Presolve time: 0.14s
Presolved: 638 rows, 1272 columns, 2544 nonzeros
Presolved model has 201894 quadratic objectiv

In [31]:
non_negative_count = np.sum(beta_msw_lasso > 0)
non_negative_count

12

# Backtest
## Experiment result 
- Turn over 비용
- Tracking error(for out of sample)
- number of selecting asset
- beta estimated


총 4개의 값 저장할것

error 없으면 time_limit을 1200으로 늘릴것(원본 논문 참고)

In [32]:
def evaluate_portfolio(X, y, beta):
    tracking_error = np.sqrt(np.mean((y - X @ beta)**2))
    turnover = np.sum(np.abs(beta[1:] - beta[:-1])) / len(beta)
    selected = np.sum(beta > 0)
    return tracking_error, turnover, selected

In [33]:
def rolling_window_backtest(stock_returns, stock_index, train_window, test_window, kact, max_iter=10, time_limit=60):
    print("----------------BenchMark TESTING--------------------")
    results = {'CARD': [], 'ALASSO': [], 'SLOPE': [], 'MSW-LASSO-SCAD': [],'MSW-LASSO-MCP': [],'MSW-LASSO-LOGM': [],'MSW-LASSO-lq': [], }
    n = len(stock_returns)
    
    for start in range(0, n - train_window - test_window + 1, test_window):
        train_data = stock_returns.iloc[start:start + train_window]
        test_data = stock_returns.iloc[start + train_window:start + train_window + test_window]
    
        # Filter out columns with NaN values in the current window
        valid_assets_train = train_data.dropna(axis=1)
        valid_assets_test = test_data.dropna(axis=1)
        valid_assets = valid_assets_train.columns.intersection(valid_assets_test.columns)
        
        X_train = valid_assets_train[valid_assets].values
        y_train = stock_index.iloc[start:start + train_window].values
        
        X_test = valid_assets_test[valid_assets].values
        y_test = stock_index.iloc[start + train_window:start + train_window + test_window].values
        # Skip if no valid assets are available
        if X_train.shape[1] == 0 or X_test.shape[1] == 0:
            continue
        
        # # CARD
        beta_card = card(X_train, y_train, kact, time_limit=time_limit)
        te_card, to_card, selected = evaluate_portfolio(X_test, y_test, beta_card)
        results['CARD'].append({'date': stock_returns.index[start + train_window], 'te': te_card, 'to': to_card, 'num of selected': selected,'beta': [beta_card] })
        
        # ALASSO
        beta_alasso = alasso(X_train, y_train, kact, time_limit=time_limit)
        te_alasso, to_alasso, selected = evaluate_portfolio(X_test, y_test, beta_alasso)
        results['ALASSO'].append({'date': stock_returns.index[start + train_window], 'te': te_alasso, 'to': to_alasso, 'num of selected': selected,'beta': [beta_alasso]})
        
        #SLOPE
        beta_slope = slope(X_train, y_train, kact, max_iter=max_iter, time_limit=time_limit)
        te_slope_slc, to_slope_slc, selected = evaluate_portfolio(X_test, y_test, beta_slope)
        results['SLOPE'].append({'date': stock_returns.index[start + train_window], 'te': te_slope_slc, 'to': to_slope_slc, 'num of selected': selected,'beta': [beta_slope]})
        
        # MSW-LASSO -SCAD
        beta_msw_lasso_scad = msw_lasso(X_train, y_train, kact=kact, penalty='SCAD', a=3.7, max_iter=max_iter, time_limit=time_limit)
        te_msw_lasso_scad, to_msw_lasso_scad, selected = evaluate_portfolio(X_test, y_test, beta_msw_lasso_scad)
        results['MSW-LASSO-SCAD'].append({'date': stock_returns.index[start + train_window], 'te': te_msw_lasso_scad, 'to': to_msw_lasso_scad, 'num of selected': selected,'beta': [beta_msw_lasso_scad]})
        
        # MSW-LASSO -MCP
        beta_msw_lasso_mcp = msw_lasso(X_train, y_train, kact=kact, penalty='MCP', a=2.5, max_iter=max_iter, time_limit=time_limit)
        te_msw_lasso_mcp, to_msw_lasso_mcp, selected = evaluate_portfolio(X_test, y_test, beta_msw_lasso_mcp)
        results['MSW-LASSO-MCP'].append({'date': stock_returns.index[start + train_window], 'te': te_msw_lasso_mcp, 'to': to_msw_lasso_mcp, 'num of selected': selected,'beta': [beta_msw_lasso_mcp]})

        # MSW-LASSO-log-M
        beta_msw_lasso_logM = msw_lasso(X_train, y_train, kact=kact, penalty='log-M', a=1.67*10**(-5), max_iter=max_iter, time_limit=time_limit)
        te_msw_lasso_logM, to_msw_lasso_logM, selected = evaluate_portfolio(X_test, y_test, beta_msw_lasso_logM)
        results['MSW-LASSO-LOGM'].append({'date': stock_returns.index[start + train_window], 'te': te_msw_lasso_logM, 'to': to_msw_lasso_logM, 'num of selected': selected,'beta': [beta_msw_lasso_logM]})

        # MSW-LASSO-lq
        beta_msw_lasso_lq = msw_lasso(X_train, y_train, kact=kact, penalty='lq', a=0.1, max_iter=max_iter, time_limit=time_limit)
        te_msw_lasso_lq, to_msw_lasso_lq, selected = evaluate_portfolio(X_test, y_test, beta_msw_lasso_lq)
        results['MSW-LASSO-lq'].append({'date': stock_returns.index[start + train_window], 'te': te_msw_lasso_lq, 'to': to_msw_lasso_lq, 'num of selected': selected,'beta': [beta_msw_lasso_lq]})

        print(f"Window ending {stock_returns.index[start + train_window].date()}:")
        print(f"CARD: TE={te_card:.4f}, TO={to_card:.4f}")
        print(f"ALASSO: TE={te_alasso:.4f}, TO={to_alasso:.4f}")
        print(f"SLOPE: TE={te_slope_slc:.4f}, TO={to_slope_slc:.4f}")
        print(f"MSW-LASSO-SCAD: TE={te_msw_lasso_scad:.4f}, TO={to_msw_lasso_scad:.4f}")
        print(f"MSW-LASSO-MCP: TE={te_msw_lasso_mcp:.4f}, TO={to_msw_lasso_mcp:.4f}")
        print(f"MSW-LASSO-logM: TE={te_msw_lasso_logM:.4f}, TO={to_msw_lasso_logM:.4f}")
        print(f"MSW-LASSO-lq: TE={te_msw_lasso_lq:.4f}, TO={to_msw_lasso_lq:.4f}")
    
    return results


In [34]:
def run_experiments(X, y, train_window, test_window, kact_values, max_iter, time_limit, save_path="../results/s&p500"):
    timestamp = datetime.now().strftime('%m%d%H%M%S')
    for kact in kact_values:
        results = rolling_window_backtest(X, y, train_window, test_window, kact, max_iter, time_limit)
        # 각 알고리즘별 결과를 데이터프레임으로 변환
        dataframes = {}
        for key, value in results.items():
            df = pd.DataFrame(value)
            df_name = f"{key}_{kact}_{timestamp}"
            dataframes[df_name] = df
    
        # 데이터프레임을 CSV 파일로 저장
        for df_name, df in dataframes.items():
            file_path = os.path.join(save_path, f"{df_name}.csv")
            df.to_csv(file_path, index=False)
            print(f"Saved {df_name} to {file_path}")
    
    return dataframes

In [35]:
# Load the stock data.
file_path = '..\data\s&p500\s&p500_2016to2022_stock_data.csv'
stock_data = pd.read_csv(file_path, index_col='date', parse_dates=True)
# Calculate daily returns
stock_returns = stock_data.pct_change().dropna(how='all')

# kact_values = [10, 20, 30, 40, 50] -> 5h 정도 걸림
kact_values = [20]

train_window = 250
test_window =21
# DATA usage
X = stock_returns.drop(columns=['^GSPC'])
y = stock_returns['^GSPC']


In [36]:
run_experiments(X, y, train_window, test_window, kact_values, max_iter=10, time_limit=60, save_path="../results/s&p500")

----------------BenchMark TESTING--------------------


# RMIT(Rank Meets Index tracking model)

1. RMIT model makes n samples of B(weight of porfolio) -> using n Indextracking model(with diffent random seed or epoch)
2. for n num of B it test tracking error using train set(most recent 1 month)
3. for n num of B it test tracking error using valid set(1 month)
4. compare rank between 2,3 and add it's loss == rank Loss + origin loss 
5. repeat 1~3 which add rank loss, stop when beta does not change(small then epsilon) 
-> for last choose best beta(smallest beta) 

### Rank LOSS

In [ ]:
def pointwise_ranking_loss(y_true, y_pred):
    """
    Pointwise ranking loss function.
    
    Parameters:
    y_true (np.array): True objective values.
    y_pred (np.array): Predicted objective values.
    
    Returns:
    float: Pointwise ranking loss.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    loss = np.mean((y_true - y_pred) ** 2)
    return loss

In [ ]:
def pairwise_ranking_loss(y_true, y_pred, margin=1.0):
    """
    Pairwise ranking loss function.
    
    Parameters:
    y_true (np.array): True objective values.
    y_pred (np.array): Predicted objective values.
    margin (float): Margin for ranking.
    
    Returns:
    float: Pairwise ranking loss.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    loss = 0.0
    n = len(y_true)
    for i in range(n):
        for j in range(n):
            if y_true[i] > y_true[j]:
                loss += max(0, margin + y_pred[j] - y_pred[i])
    return loss / (n * (n - 1))

In [ ]:
def pairwise_difference_ranking_loss(y_true, y_pred):
    """
    Pairwise difference ranking loss function.
    
    Parameters:
    y_true (np.array): True objective values.
    y_pred (np.array): Predicted objective values.
    
    Returns:
    float: Pairwise difference ranking loss.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    loss = 0.0
    n = len(y_true)
    for i in range(n):
        for j in range(n):
            loss += (y_true[i] - y_true[j] - (y_pred[i] - y_pred[j])) ** 2
    return loss / (n * (n - 1))


In [ ]:
def listwise_ranking_loss(y_true, y_pred, tau=1.0):
    """
    Listwise ranking loss function.
    
    Parameters:
    y_true (np.array): True objective values.
    y_pred (np.array): Predicted objective values.
    tau (float): Temperature parameter.
    
    Returns:
    float: Listwise ranking loss.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    def softmax(x, tau=1.0):
        e_x = np.exp(x / tau)
        return e_x / e_x.sum()

    true_prob = softmax(-y_true, tau)
    pred_prob = softmax(-y_pred, tau)
    loss = -np.sum(true_prob * np.log(pred_prob))
    return loss


In [ ]:

def rank_loss(rankloss_type, y_true, y_pred,):
    """
    Return the appropriate ranking loss function based on the provided rankloss type.
    
    Parameters:
    rankloss (str): Type of ranking loss ('pointwise', 'pairwise', 'listwise').
    
    Returns:
    rank error rank.
    """
    if rankloss_type == 'pointwise':
        return pointwise_ranking_loss(y_true, y_pred)
    elif rankloss_type == 'pairwise':
        return pairwise_ranking_loss(y_true, y_pred, margin=1.0)
    elif rankloss_type == 'pairwise-diff':
        return pairwise_difference_ranking_loss(y_true, y_pred)
    elif rankloss_type == 'listwise':
        return listwise_ranking_loss(y_true, y_pred, tau=1.0)
    else:
        raise ValueError("Unsupported rankloss type. Choose from 'pointwise', 'pairwise', 'listwise'.")

### sampling Beta

In [ ]:
def slope_sampling(X, y, kact, alpha=1.0, theta=0.1, tol=1e-06, max_iter=10, time_limit=1200, add_loss=0, seed=42):
    n, p = X.shape
    beta_sample = []
    weights = np.ones(p)
    m = gp.Model()
    m.setParam('TimeLimit', time_limit)
    m.setParam(GRB.Param.Seed, seed)
    beta = m.addMVar(shape=p, lb=0, name="beta")
    z = m.addMVar(shape=p, vtype=GRB.BINARY, name="z")
    m.addConstr(beta <= z, name="c0")
    m.addConstr(beta.sum() == 1, name="c1")
    m.addConstr(z.sum() <= kact, name="c2")
    for _ in range(max_iter):
        m.reset()   
        objective = (1/n) * ((y - X @ beta) @ (y - X @ beta)) + weights @ beta + add_loss
        m.setObjective(objective, GRB.MINIMIZE)
        m.optimize()
        # 결과 출력
        if m.status == GRB.OPTIMAL or m.status == GRB.SUBOPTIMAL:
            beta_value = beta.X
            beta_sample.append(beta_value)
            new_weights = np.array(new_lambda(beta_value, alpha, theta, p))
            weights = new_weights
            
            if np.linalg.norm(new_weights - weights) < tol:
                continue
        else:
            beta_value = np.zeros(p)
            beta_sample.append(beta_value)
            continue
    m.dispose()        
    return beta_sample


def msw_lasso_sampling(X, y, kact, max_iter=10, tol=1e-12, penalty='MCP', a=2.5, time_limit = 1200, add_loss=0, seed=42):
    n, p = X.shape
    weights = np.ones(p)
    beta_sample = []
    def p_lambda(beta_j, lam, a):
        #MCP: b, a =2.5
        #SCAD: a=3.7
        #log-M: epsilon = 1.67*10^-5
        #lq: q=0.1
        if penalty == 'MCP':
            b = a 
            return lam * beta_j - (beta_j**2) / (2 * b) if beta_j <= b * lam else (b * lam**2) / 2
        elif penalty == 'SCAD':
            if beta_j <= lam:
                return lam * beta_j
            elif beta_j <= a * lam:
                return (-beta_j**2 + 2 * a * lam * beta_j - lam**2) / (2 * (a - 1))
            else:
                return (a + 1) * lam**2 / 2
        elif penalty == 'log-M':
            eps = a
            return lam * np.log(1+np.abs(beta_j)/eps) / np.log(1+(1/eps))
        elif penalty == 'lq':
            q = a 
            return lam * (abs(beta_j) **q) 
    m = gp.Model()
    m.setParam('TimeLimit', time_limit)
    m.setParam(GRB.Param.Seed, seed)
    beta = m.addMVar(shape=p, lb=0, name="beta")
    z = m.addMVar(shape=p, vtype=GRB.BINARY, name="z")
    m.addConstr(beta <= z, name="c0")
    m.addConstr(beta.sum() == 1, name="c1")
    m.addConstr(z.sum() <= kact, name="c2")
    
    for _ in range(max_iter):
        m.reset()   
        objective = (1/n) * ((y - X @ beta) @ (y - X @ beta)) + weights @ beta + add_loss
        m.setObjective(objective, GRB.MINIMIZE)
        m.optimize()      
        # 결과 출력
        if m.status == GRB.OPTIMAL or m.status == GRB.SUBOPTIMAL:
            beta_value = beta.X
            beta_sample.append(beta_value)
            new_weights = np.array([abs(p_lambda(beta_value[j], 1, a)) for j in range(p)])
            weights = new_weights
            
            if np.linalg.norm(new_weights - weights) < tol:
                continue
        else:
            beta_value = np.zeros(p)
            beta_sample.append(beta_value)
            continue
    m.dispose()         
    return beta_sample

In [ ]:
def sampling_Beta(X, y, kact, algorithm_name, num_sample, time_limit, add_loss=0):
# msw_lasso(X_train, y_train, kact=kact, penalty='lq', a=0.1, max_iter=max_iter, time_limit=time_limit)
#beta_slope = slope(X_train, y_train, kact, max_iter=max_iter, time_limit=time_limit)
# stock_returns, stock_index, train_window, test_window, kact, max_iter=10, time_limit=1200
    if algorithm_name == 'SLOPE':       
        #SLOPE
        beta_sample = slope_sampling(X_train, y_train, kact, max_iter=num_sample, time_limit=time_limit, add_loss=add_loss)
    elif algorithm_name == 'MSW-LASSO-SCAD':   
        # MSW-LASSO -SCAD
        beta_sample = msw_lasso_sampling(X_train, y_train, kact=kact, penalty='SCAD', a=3.7, max_iter=num_sample, time_limit=time_limit, add_loss=add_loss)
    elif algorithm_name == 'MSW-LASSO-MCP':      
        # MSW-LASSO -MCP
        beta_sample = msw_lasso_sampling(X_train, y_train, kact=kact, penalty='MCP', a=2.5, max_iter=num_sample, time_limit=time_limit, add_loss=add_loss)
    elif algorithm_name == 'MSW-LASSO-logM':  
        # MSW-LASSO-log-M
        beta_sample = msw_lasso_sampling(X_train, y_train, kact=kact, penalty='log-M', a=1.67*10**(-5), max_iter=num_sample, time_limit=time_limit, add_loss=add_loss)
    elif algorithm_name == 'MSW-LASSO-lq':  
        # MSW-LASSO-lq
        beta_sample = msw_lasso_sampling(X_train, y_train, kact=kact, penalty='lq', a=0.1, max_iter=num_sample, time_limit=time_limit, add_loss=add_loss)
    
    return beta_sample

In [ ]:
def calculate_Beta(X, y, kact, algorithm_name, max_iter, time_limit, add_loss=0):
# msw_lasso(X_train, y_train, kact=kact, penalty='lq', a=0.1, max_iter=max_iter, time_limit=time_limit)
#beta_slope = slope(X_train, y_train, kact, max_iter=max_iter, time_limit=time_limit)
# stock_returns, stock_index, train_window, test_window, kact, max_iter=10, time_limit=1200
    if algorithm_name == 'SLOPE':       
        #SLOPE
        beta_sample = slope(X_train, y_train, kact, max_iter=max_iter, time_limit=time_limit)
    elif algorithm_name == 'MSW-LASSO-SCAD':   
        # MSW-LASSO -SCAD
        beta_sample = msw_lasso(X_train, y_train, kact=kact, penalty='SCAD', a=3.7, max_iter=max_iter, time_limit=time_limit, add_loss=add_loss)
    elif algorithm_name == 'MSW-LASSO-MCP':      
        # MSW-LASSO -MCP
        beta_sample = msw_lasso(X_train, y_train, kact=kact, penalty='MCP', a=2.5, max_iter=max_iter, time_limit=time_limit, add_loss=add_loss)
    elif algorithm_name == 'MSW-LASSO-logM':  
        # MSW-LASSO-log-M
        beta_sample = msw_lasso(X_train, y_train, kact=kact, penalty='log-M', a=1.67*10**(-5), max_iter=max_iter, time_limit=time_limit, add_loss=add_loss)
    elif algorithm_name == 'MSW-LASSO-lq':  
        # MSW-LASSO-lq
        beta_sample = msw_lasso(X_train, y_train, kact=kact, penalty='lq', a=0.1, max_iter=max_iter, time_limit=time_limit, add_loss=add_loss)
    
    return beta_sample

In [ ]:
def RMIT(X, y, test_window_size=21, kact=10, algorithm='SLOPE', rank_type='pointwise', num_sample=10, max_iter=10, time_limit=60):
    beta = None
    rank_loss_value = 0 
    X_train = X[:-2*test_window_size]
    y_train = y[:-2*test_window_size]
    X_train_last = X[-2*test_window_size:-test_window_size]
    y_train_last = y[-2*test_window_size:-test_window_size]
    X_val = X[-test_window_size:]
    y_val = y[-test_window_size:]
    
    for _ in range(max_iter-1): 
        # 1. Generate n samples of B using n regression models & use updated rank_loss 
        B_samples = sampling_Beta(X_train, y_train, kact, algorithm_name=algorithm, num_sample=num_sample, time_limit=time_limit, add_loss = rank_loss_value)
        # 2. Test tracking error using train set(1month)
        # 3. Test tracking error using valid set
        train_errors = []
        valid_errors = []
        for B in B_samples:
            te, _ = evaluate_portfolio(X_train_last, y_train_last, B)
            train_errors.append(te)
            te_v, _ = evaluate_portfolio(X_val, y_val, B)
            valid_errors.append(te_v)
        # 4. Compare rank between train and valid errors and add its loss
        rank_loss_value = rank_loss(rank_type, train_errors, valid_errors)

    #5. compute last beta using rankloss + origin loss of algorithm 
    # use all data + val
    beta = calculate_Beta(X, y, kact, algorithm_name=algorithm, max_iter=max_iter, time_limit=time_limit, add_loss = rank_loss_value)

    return beta


In [ ]:
stock_stocks = stock_returns.drop(columns=['^GSPC'])
stock_index = stock_returns['^GSPC']

train_data = stock_stocks.iloc[:250]

# Filter out columns with NaN values in the current window
valid_assets_train = train_data.dropna(axis=1)
valid_assets = valid_assets_train.columns.intersection(valid_assets_train.columns)

X_train = valid_assets_train[valid_assets].values
y_train = stock_index.iloc[:250].values

In [ ]:
## RMIT
beta_RMIT = RMIT(X_train, y_train, test_window_size=21, kact=10,  algorithm='SLOPE', rank_type='pointwise', num_sample=10, max_iter=10, time_limit=60)
non_negative_count = np.sum(beta_card > 0)

In [ ]:
non_negative_count

In [ ]:
## RMIT
beta_RMIT = RMIT(X_train, y_train, test_window_size=21, kact=10,  algorithm='SLOPE', rank_type='pairwise', num_sample=10, max_iter=10, time_limit=60)
non_negative_count = np.sum(beta_card > 0)

In [ ]:
non_negative_count

In [ ]:
## RMIT
beta_RMIT = RMIT(X_train, y_train, test_window_size=21, kact=10,  algorithm='SLOPE', rank_type='pairwise-diff', num_sample=10, max_iter=10, time_limit=60)
non_negative_count = np.sum(beta_card > 0)

In [ ]:
non_negative_count

In [ ]:
## RMIT
beta_RMIT = RMIT(X_train, y_train, test_window_size=21, kact=10,  algorithm='SLOPE', rank_type='listwise', num_sample=10, max_iter=10, time_limit=60)
non_negative_count = np.sum(beta_card > 0)

In [ ]:
non_negative_count

# Experiment

In [ ]:
def rolling_window_backtest_RMIT(stock_returns, stock_index, train_window, test_window, kact, rank_type='pointwise', num_sample=10, max_iter=10, time_limit=60):
    print("----------------Rank Meets Index tracking TESTING--------------------")
    results = {'RMIT-SLOPE': [], 'RMIT-MSW-LASSO-SCAD': [],'RMIT-MSW-LASSO-MCP': [],'RMIT-MSW-LASSO-LOGM': [],'RMIT-MSW-LASSO-lq': [], }
    n = len(stock_returns)
    
    for start in range(0, n - train_window - test_window + 1, test_window):
        train_data = stock_returns.iloc[start:start + train_window]
        test_data = stock_returns.iloc[start + train_window:start + train_window + test_window]
    
        # Filter out columns with NaN values in the current window
        valid_assets_train = train_data.dropna(axis=1)
        valid_assets_test = test_data.dropna(axis=1)
        valid_assets = valid_assets_train.columns.intersection(valid_assets_test.columns)
        
        X_train = valid_assets_train[valid_assets].values
        y_train = stock_index.iloc[start:start + train_window].values
        
        X_test = valid_assets_test[valid_assets].values
        y_test = stock_index.iloc[start + train_window:start + train_window + test_window].values
        # Skip if no valid assets are available
        if X_train.shape[1] == 0 or X_test.shape[1] == 0:
            continue
        
        # RMIT-SLOPE
        beta_RMIT_SLOPE = RMIT(X_train, y_train, test_window_size=test_window, kact=kact,  algorithm='SLOPE', rank_type=rank_type, num_sample=num_sample, max_iter=max_iter, time_limit=time_limit)
        te_SLOPE, to_SLOPE, selected = evaluate_portfolio(X_test, y_test, beta_RMIT_SLOPE)
        results['RMIT-SLOPE'].append({'date': stock_returns.index[start + train_window], 'te': te_SLOPE, 'to': to_SLOPE, 'num of selected': selected, 'beta':[beta_RMIT_SLOPE]})
        
        # RMIT-MSW-LASSO-SCAD
        beta_RMIT_LASSO_SCAD = RMIT(X_train, y_train, test_window_size=test_window, kact=kact,  algorithm='MSW-LASSO-SCAD', rank_type=rank_type, num_sample=num_sample, max_iter=max_iter, time_limit=time_limit)
        te_SCAD, to_SCAD, selected = evaluate_portfolio(X_test, y_test, beta_RMIT_LASSO_SCAD )
        results['RMIT-MSW-LASSO-SCAD'].append({'date': stock_returns.index[start + train_window], 'te': te_SCAD, 'to': to_SCAD, 'num of selected': selected, 'beta':[beta_RMIT_LASSO_SCAD]})
        
        #'RMIT-MSW-LASSO-MCP'
        beta_RMIT_LASSO_MCP = RMIT(X_train, y_train, test_window_size=test_window, kact=kact,  algorithm='MSW-LASSO-MCP', rank_type=rank_type, num_sample=num_sample, max_iter=max_iter, time_limit=time_limit)
        te_MCP, to_MCP, selected = evaluate_portfolio(X_test, y_test, beta_RMIT_LASSO_MCP)
        results['RMIT-MSW-LASSO-MCP'].append({'date': stock_returns.index[start + train_window], 'te': te_MCP, 'to': to_MCP, 'num of selected': selected, 'beta':[beta_RMIT_LASSO_MCP]})
        
        # RMIT-MSW-LASSO-LOGM
        beta_RMIT_LASSO_LOGM = RMIT(X_train, y_train, test_window_size=test_window, kact=kact,  algorithm='MSW-LASSO-logM', rank_type=rank_type, num_sample=num_sample, max_iter=max_iter, time_limit=time_limit)
        te_LOGM, to_LOGM, selected = evaluate_portfolio(X_test, y_test, beta_RMIT_LASSO_LOGM )
        results['RMIT-MSW-LASSO-LOGM'].append({'date': stock_returns.index[start + train_window], 'te': te_LOGM, 'to': to_LOGM, 'num of selected': selected, 'beta':[beta_RMIT_LASSO_LOGM]})
        
        # RMIT-MSW-LASSO-lq
        beta_RMIT_LASSO_lq = RMIT(X_train, y_train, test_window_size=test_window, kact=kact,  algorithm='MSW-LASSO-lq', rank_type=rank_type, num_sample=num_sample, max_iter=max_iter, time_limit=time_limit)
        te_lq, to_lq, selected = evaluate_portfolio(X_test, y_test, beta_RMIT_LASSO_lq)
        results['RMIT-MSW-LASSO-lq'].append({'date': stock_returns.index[start + train_window], 'te': te_lq, 'to': to_lq, 'num of selected': selected, 'beta':[beta_RMIT_LASSO_lq]})

        print(f"Window ending {stock_returns.index[start + train_window].date()}:")
        print(f"RMIT-SLOPE: TE={te_SLOPE:.4f}, TO={to_SLOPE:.4f}")
        print(f"RMIT-MSW-LASSO-SCAD: TE={te_SCAD:.4f}, TO={to_SCAD:.4f}")
        print(f"RMIT-MSW-LASSO-MCP: TE={te_MCP:.4f}, TO={te_MCP:.4f}")
        print(f"RMIT-MSW-LASSO-logM: TE={te_LOGM:.4f}, TO={to_LOGM:.4f}")
        print(f"RMIT-MSW-LASSO-lq: TE={te_lq:.4f}, TO={to_lq:.4f}")
    
    return results

In [ ]:
# Load the stock data.
file_path = '..\data\s&p500\s&p500_2016to2022_stock_data.csv'
stock_data = pd.read_csv(file_path, index_col='date', parse_dates=True)
# Calculate daily returns
stock_returns = stock_data.pct_change().dropna(how='all')
# kact_values = [10, 20, 30, 40, 50] -> 5h 정도 걸림
kact_values = [10]
# sampling_size = [3, 7, 10] 
sampling_size = [3] 

ranking_types = ['pointwise', 'pairwise','pairwise-diff', 'listwise']

train_window = 250
test_window =21
# sampling_size = [10]
# DATA usage
X = stock_returns.drop(columns=['^GSPC'])
y = stock_returns['^GSPC']


In [ ]:
def run_rmit_experiments(X, y, train_window, test_window, kact_values, num_sample_lst, ranking_lst, max_iter, time_limit, save_path="../results/s&p500"):
    timestamp = datetime.now().strftime('%m%d%H%M%S')
    for kact in kact_values:
        for num_sample in num_sample_lst:
            for rank_type in ranking_lst:
                results = rolling_window_backtest_RMIT(X, y, train_window, test_window, kact, rank_type, num_sample, max_iter, time_limit)
                # 각 알고리즘별 결과를 데이터프레임으로 변환
                dataframes = {}
                for key, value in results.items():
                    df = pd.DataFrame(value)
                    df_name = f"{key}_{kact}_{timestamp}"
                    dataframes[df_name] = df
        
                # 데이터프레임을 CSV 파일로 저장
                for df_name, df in dataframes.items():
                    file_path = os.path.join(save_path, f"{df_name}.csv")
                    df.to_csv(file_path, index=False)
                    print(f"Saved {df_name} to {file_path}")
    
    return dataframes

In [ ]:
run_rmit_experiments(X, y, train_window, test_window, kact_values, max_iter=10, num_sample_lst = sampling_size , time_limit=60, save_path="../results/s&p500")

In [ ]:
# df_results = pd.read_csv('../results/s&p500_MSW-LASSO-MCP_kact_10.csv')
# df_results.head()

# df_results = pd.read_csv('../results_s&p500_kact_20.csv')
# # Plot the results
# plt.figure(figsize=(14, 7))

# for key in df_results:
#     plt.plot(df_results[key]['te'], label=f'{key} Tracking Error')

# plt.title('Tracking Error Over Time')
# plt.xlabel('Date')
# plt.ylabel('Tracking Error')
# plt.legend()
# plt.show()

# plt.figure(figsize=(14, 7))

# for key in df_results:
#     plt.plot(df_results[key]['to'], label=f'{key} Turnover')

# plt.title('Turnover Over Time')
# plt.xlabel('Date')
# plt.ylabel('Turnover')
# plt.legend()
# plt.show()
